In [1]:
import time
import numpy as np
import os
import OSEA
import pandas as pd
import biom
import heatmap

In [2]:
taxonomy_filename = os.path.join('','upload_files/taxonomy.tsv')

feature_table = os.path.join('','upload_files/feature-table.biom')


In [3]:

metadata_filename = os.path.join('', 'upload_files/demo/demo_metadata.tsv')
#with open(metadata_filename) as metadata_file:
    #metadata = pd.read_csv(metadata_file, sep='\t') 

In [4]:
heatmap_instance = heatmap.Heatmap(metadata_filename,feature_table)
heatmap_instance.map()

In [5]:
import stats_test
part1, part2 = stats_test.choose_two_class(heatmap_instance.df,'Subject')
part1 = part1[heatmap_instance.df_primary_col]
part2 = part2[heatmap_instance.df_primary_col]
def obtain_rank_list(df1, df2, test_method_name='t_test'):
    methods = {'t_test': stats_test.t_test,
               'F_test': stats_test.F_test}
    test_method = methods[test_method_name]
    rank_list_unsort = {}
    for col in part1.columns:
        tmp_pvalue = test_method(part1[col],part2[col])
        rank_list_unsort[col]=tmp_pvalue
    tmp_list = sorted(rank_list_unsort, key=rank_list_unsort.get,reverse=True)
    rank_list = {}
    for ele in tmp_list:
        rank_list[ele]=rank_list_unsort[ele]
    return rank_list

rank_list = obtain_rank_list(part1,part2)

#rank_list

osea_real = OSEA.OSEA(rank_list,Taxon_file=taxonomy_filename,set_level='Genus')

In [6]:
enrichment_scores = []
tic = time.time()
first_tic = time.time()
tmp_df = heatmap_instance.df[heatmap_instance.df_primary_col]
for i in range(1000):
    #print('#######____'+str(i)+'___####')
    rank_list = OSEA.permutation_to_obtain_ranklist(tmp_df)
    es = osea_real.get_ES(rank_list)
    enrichment_scores.append(es)
    #osea.es
    new_tic = time.time()
    #print('the %d th permutation used %d seconds'%(i,new_tic-tic))
    tic = new_tic
new_tic = time.time()
#print('the total time used is %.4f'%(new_tic-first_tic))

In [7]:
len(enrichment_scores)

1000

In [8]:
print('the time used is %.5f seconds'%(new_tic-first_tic))

the time used is 340.75284 seconds


In [9]:
#for i in range(100):
set_enrichment_score = {}
for key in enrichment_scores[0]:
    set_enrichment_score[key] = []
    for i in range(1000):
        set_enrichment_score[key].append(enrichment_scores[i][key])


i = 1
for ele in set_enrichment_score:
    plt.subplot(5,4,i)
    plt.hist(set_enrichment_score[ele],bins=25)
    i += 1
plt.show()

In [10]:
osea_real.es

{' g__Actinomyces': 0.3161829375549692,
 ' g__': 0.11218431963814512,
 ' g__Prevotella': 0.10026385224274426,
 ' g__Neisseria': 0.30606860158311344,
 ' g__Acinetobacter': 0.40953637391632103,
 ' g__Veillonella': 0.148636763412489,
 ' g__Subdoligranulum': 0.6490765171503958,
 ' g__Porphyromonas': 0.21487467018469658,
 ' g__Oligella': 0.07783641160949872,
 ' g__Pseudomonas': 0.29683377308707126,
 ' g__Cloacibacterium': 0.9538258575197889,
 ' g__Fusobacterium': 0.10861917326297273,
 ' g__Lactobacillus': 0.16754617414248008,
 ' g__Mycoplasma': 0.5017590149516271,
 ' g__Shuttleworthia': 0.3720316622691293,
 ' g__Oribacterium': 0.01846965699208447,
 ' g__Paracoccus': 0.5461741424802111,
 ' g__Flavobacterium': 0.21108179419525053,
 ' g__Acanthamoeba': 0.4036939313984169,
 ' g__Oxalobacter': 0.4617414248021108,
 ' g__Coprococcus': 0.1299472295514512,
 ' g__Treponema': 0.21459982409850475,
 ' g__Lachnospira': 0.11948737278552579,
 ' g__Faecalibacterium': 0.13324538258575197,
 ' g__Bacteroides':

In [11]:
final_result = {}
for ele in set_enrichment_score:
    distribution = OSEA.generate_distribution(set_enrichment_score[ele])
    pvalue = OSEA.p_value(osea_real.es[ele],distribution)
    final_result[ele]={'ES':osea_real.es[ele],'pvalue':pvalue}

In [12]:
i = 0
for ele in final_result:
    if final_result[ele]['pvalue'] < 0.05:
        print(ele,final_result[ele])
        i+=1
print(i)

 g__Subdoligranulum {'ES': 0.6490765171503958, 'pvalue': 0.04700000000000003}
 g__Clostridium {'ES': 0.47950071037142283, 'pvalue': 0.009000000000000001}
 g__Helcococcus {'ES': 0.7427440633245382, 'pvalue': 0.016000000000000004}
 g__Holdemania {'ES': 0.9775725593667546, 'pvalue': 0.041}
 g__Methylobacterium {'ES': 0.7704485488126649, 'pvalue': 0.003}
 g__Rhodococcus {'ES': 0.9934036939313984, 'pvalue': 0.02}
 g__Bergeyella {'ES': 0.7994722955145119, 'pvalue': 0.044000000000000025}
 g__Janthinobacterium {'ES': 0.7902374670184696, 'pvalue': 0.003}
 g__Blastococcus {'ES': 0.9947229551451188, 'pvalue': 0.027999999999999997}
 g__Arcobacter {'ES': 0.7532981530343008, 'pvalue': 0.003}
 g__Alicyclobacillus {'ES': 0.996042216358839, 'pvalue': 0.02}
 g__Filifactor {'ES': 0.7572559366754618, 'pvalue': 0.005}
 g__Desulfomicrobium {'ES': 0.7361477572559367, 'pvalue': 0.023000000000000013}
 g__Luteolibacter {'ES': 0.7440633245382586, 'pvalue': 0.02500000000000001}
 g__Cellvibrio {'ES': 0.75197889182